## Joins with `sqlalchemy`

To join to tables in `sqlalchemy`

1. Use `join` to create a `Join` object
2. Build a `select` statement from the `join` object

## Example - Reading in the Company `db`

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite:///databases/company_2_7_2.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Dept = Base.classes.department
Empl = Base.classes.employee

In [ ]:
import pandas as pd
from sqlalchemy import select as selectq
d = selectq([Dept])
pd.read_sql_query(d, con=engine)

In [ ]:
e = selectq([Empl])
pd.read_sql_query(e, con=engine)

## Using `sqlalchemy.join` to create a `Join`

**Syntax:** `join(left_table, right_table, onclause=left_table.column == right_table.column)`

* Defaults to an inner join
* Set `isouter=True` to get a `LEFT OUTER JOIN`
* Set `full=True` to get a `FULL OUTER JOIN`

In [ ]:
from sqlalchemy import join
j = join(Empl, Dept, onclause=Empl.DeptID == Dept.DeptID)
print(j)

## Inspecting the joined column names

Note that the columns are renamed `tableName_columnName`.

In [ ]:
j.c.keys()

## Creating a `Select` expression for a `Join`

In [ ]:
from sqlalchemy import func, select as selectq

stmt = selectq([j])
print(stmt)

In [ ]:
pd.read_sql_query(stmt, con=engine)

## Left Join

In [ ]:
left_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
left_join_stmt = selectq([left_join])
pd.read_sql_query(left_join_stmt, con=engine)

## Right Join

To get a `RIGHT OUTER JOIN`, just switch the order and use a `LEFT OUTER JOIN`

In [ ]:
right_join = join(Dept, Empl, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
right_join_stmt = selectq([right_join])
pd.read_sql_query(right_join_stmt, con=engine)

## Full Outer Join

**Note:** `sqllite` does not support this type of join `:/`

In [ ]:
full_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, full=True)
full_join_stmt = selectq([full_join])
pd.read_sql_query(full_join_stmt, con=engine)

## <font color="red"> Exercise 3 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, you will need to create a database containing these three tables and use the `sqlalchemy` join methods presented above.

In [1]:
files = ("./data/baseball/core/Batting.csv", 
              "./data/baseball/core/People.csv",
              "./data/baseball/core/Teams.csv")

In [2]:
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import select



In [3]:
from sqlalchemy import Integer, Float, String, DateTime
DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}

In [4]:
batting, people, teams = [pd.read_csv(x) for x in files]

In [5]:
def get_sql_types(df):
    sql_type = lambda dtype: DTYPES_TO_SQLALCHEMY_TYPES[dtype.kind] 
    cols_and_dtypes = lambda df: zip(df.columns, df.dtypes)
    return {col:sql_type(dtype) 
            for col, dtype in cols_and_dtypes(df)}

In [8]:
people_sqltypes = get_sql_types(people)
teams_sqltypes = get_sql_types(teams)
batting_sqltypes = get_sql_types(batting)

In [16]:
mang_eng = create_engine("sqlite:///databases/baseball_1_6.db")
mang_eng.echo = False
people_schema = pd.io.sql.get_schema(people, 'people', keys='playerID', con=mang_eng, dtype=people_sqltypes)
teams_schema = pd.io.sql.get_schema(teams, 'teams', con=mang_eng, dtype=teams_sqltypes)
batting_schema = pd.io.sql.get_schema(batting, 'batting',con=mang_eng, dtype=batting_sqltypes)

print(people_schema)
print(teams_schema)
print(batting_schema)


CREATE TABLE people (
	"playerID" VARCHAR NOT NULL, 
	"birthYear" FLOAT, 
	"birthMonth" FLOAT, 
	"birthDay" FLOAT, 
	"birthCountry" VARCHAR, 
	"birthState" VARCHAR, 
	"birthCity" VARCHAR, 
	"deathYear" FLOAT, 
	"deathMonth" FLOAT, 
	"deathDay" FLOAT, 
	"deathCountry" VARCHAR, 
	"deathState" VARCHAR, 
	"deathCity" VARCHAR, 
	"nameFirst" VARCHAR, 
	"nameLast" VARCHAR, 
	"nameGiven" VARCHAR, 
	weight FLOAT, 
	height FLOAT, 
	bats VARCHAR, 
	throws VARCHAR, 
	debut VARCHAR, 
	"finalGame" VARCHAR, 
	"retroID" VARCHAR, 
	"bbrefID" VARCHAR, 
	CONSTRAINT people_pk PRIMARY KEY ("playerID")
)



CREATE TABLE teams (
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	"teamID" VARCHAR, 
	"franchID" VARCHAR, 
	"divID" VARCHAR, 
	"Rank" INTEGER, 
	"G" INTEGER, 
	"Ghome" FLOAT, 
	"W" INTEGER, 
	"L" INTEGER, 
	"DivWin" VARCHAR, 
	"WCWin" VARCHAR, 
	"LgWin" VARCHAR, 
	"WSWin" VARCHAR, 
	"R" INTEGER, 
	"AB" INTEGER, 
	"H" INTEGER, 
	"2B" INTEGER, 
	"3B" INTEGER, 
	"HR" INTEGER, 
	"BB" FLOAT, 
	"SO" FLOAT, 
	"SB" F

In [10]:
mang_eng.execute(teams_schema)
mang_eng.execute(batting_schema)
mang_eng.execute(people_schema)

In [17]:
people.to_sql(name = 'people',
               con = mang_eng, 
               if_exists = 'append',
               index = False,
               dtype = people_sqltypes
              )

In [12]:
batting.to_sql(name = 'batting',
               con = mang_eng, 
               if_exists = 'append',
               index = False,
               dtype = batting_sqltypes
              )

In [13]:
teams.to_sql(name = 'teams',
               con = mang_eng, 
               if_exists = 'append',
               index = False,
               dtype = teams_sqltypes
             )

In [14]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball_1_7.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [18]:
Base = automap_base()
Base.prepare(mang_eng2, reflect = True)
People = Base.classes.people
#teams = Base.classes.teams
#batting = Base.classes.batting

AttributeError: people

In [ ]:
battingTbl.__table__.drop()
teamsTbl.__table__.drop()
PeopleTbl.__table__.drop()

In [ ]:
battingTbl = batting.__table__
teamsTbl = teams.__table__
PeopleTbl = People.__table__




In [ ]:
from sqlalchemy import union, union_all, intersect, intersect_all, except_, except_all
from sqlalchemy import select as selectq


In [ ]:
pd.read_sql_query(selectq([People]), con=mang_eng2)

In [ ]:
pd.read_sql_query(selectq([teams]), con=mang_eng2)

In [ ]:
pd.read_sql_query(selectq([batting]), con=mang_eng2)

In [ ]:
# Your code here

engine = create_engine("sqlite:///databases/test.db", echo = False)
Base = automap_base()
Base.prepare(engine, reflect=True)
Session = sessionmaker(engine)
session = Session()

people = Base.classes.people

In [ ]:
Base.classes.people

In [ ]:
batting = pd.read_csv(files[1])

In [ ]:
batting_table = Base.classes.batting

In [ ]:
batting_table

In [ ]:
engine = create_engine('sqlite:///batting.db')
Base.metadata.create_all(engine)

df = pd.read_csv(f1)
df.to_sql('batting',
        con=engine, 
        index = False, 
        if_exists='append')

## Up Next

Stuff